# 사용자 정의 모델
**keras.Model** 클래스를 상속 
> - **생성자**에서 층과 변수를 만들고 
>- 모델이 해야 할 작업을 **call** 메서드에 구현

In [1]:
import tensorflow as tf
from tensorflow import keras

In [5]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                         kernel_initializer="he_normal") for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 
               "thr": self.thr}

**residual block** 으로 이루어진 네트워크를 만들기 위해 사전에 레지듀얼 블록을 정의  
이제 서브클래싱 API를 이용해 모델을 정의해보겠음

In [6]:
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                         kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
        
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 
               "thr": self.thr}

이제 위 모델은 다른 일반 모델들처럼 사용할 수 있음.  
> - **save()** 메서드를 사용해 모델을 저장하고 **keras.models.load_model()** 함수를 이용해 저장된 모델을 로드하고 싶으면 RedisualBlock 클래스와 ResidualRegressor 클래스에 모두 **get_config()** 메서드를 구현해야 함.

또한 **save_weights()와 load_weights()** 메서드를 사용해 가중치를 저장하고 로드할 수 있음.

In [12]:
temp_model = ResidualRegressor(10)

'''model.compile(loss="mse", optimizer="nadam")
hist = model.fit(x_train, y_train, epoch=5)
file_path = "./temp_model.ckpt"
temp_model.save(file_path)

loaded_model = keras.models.load_model(file_path)'''

'model.compile(loss="mse", optimizer="nadam")\nhist = model.fit(x_train, y_train, epoch=5)\nfile_path = "./temp_model.ckpt"\ntemp_model.save(file_path)\n\nloaded_model = keras.models.load_model(file_path)'

Model 클래스는 Layer클래스이 서브클래스임.  
- 따라서 모델을 층처럼 정의할 수 있음.  
- 그 외에도 compile(), fit(), evaluate(), predict() 와 같은 추가적인 기능도 있음. 그리고 get_layers(), save()도 있음.  


---
# 모델 구성 요소에 따른 손실과 지표
앞서 정의한 사용자 손실과 지표는 모두 레이블과 예측(그리고 샘플 가중치)을 기반으로 함.  
은닉층의 가중치나 활성 함수 등과 같이 모델의 구성 요소에 기반한 손실을 정의해야 할 때가 있음.
> 이런 손실은 규제나 모델의 내부 상황을 모니터링할 때 유용함.

모델 구성 요소에 기반한 손실을 정의하고 계산하여 **add_loss()** 메서드에 그 결과를 전달함.

In [20]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                        kernel_initializer="lecun_normal") for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        
        self.reconstruct = keras.layers.Dense(8)
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")
        
    '''def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")
        super().build(batch_input_shape)'''
    
    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)

**재구성 손실 (reconstruction loss)** 를 손실에 전달하는 모델 예시 (오토엔코더같음)  
재구성 손실을 주 손실에 더하여 회귀작어버에 직접적으로 도움이 되지 않은 정보일지라도 모델이 은닉층을 통과하면서  
> 가능한 많은 정보를 유지하도록 유도함  
이러한 손실은 이따금 일반화 성능을 향상시킨다고 함 (규제처럼)

- build() 메서드에서 풀리 커넥티드 층을 하나 더 추가하여 모델의 입력을 재구성하는 데 사용. 이 풀리커넥티드 층의 유닛수는 입력 개수와 같아야 함. **이 재구성층을 build()에서 만드는 이유는** 이 메서드가 호출되기 전까지는 입력 개수를 알 수 없기 때문임.
- call() 메서드에서 결괏값을 재구성층에 전달하여 재구성을 만듦. 재구성 손실을 계산하고 **add_loss()** 메서드를 사용해 모델의 손실 리스트에 추가함. **이때 주 손실을 압도하지 않도록 0.05같은(하이퍼파라미터로 튜닝할 값) 작은 수를 곱함.**

이와 비슷하게 모델의 구성 요소에 기반하여 임의의 계산을 수행하는 사용자 정의 지표를 추가할 수 있음.
> 다만 결과값이 지표 객체의 출력이어야 함(???)

예를 들면 keras.metrics.Mean 객체를 만들고 recon_loss를 전달하면서 call()메서드를 호출할 수 있음.  
>마지막으로 모델의 **add_metric()** 메서드를 호출하여 모델에 이 지표를 추가함. 이렇게 하면 모델을 학습할 때 케라스가 에포크마다 평균 손실과 평균 재구성 손실을 출력함.

In [15]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [21]:
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 [==============================] - 0s 535us/step - loss: 0.7520 - reconstruction_error: 0.7959
Epoch 2/2
363/363 [==============================] - 0s 501us/step - loss: 0.4131 - reconstruction_error: 0.3210


사용자 정의 손실을 정의하고 학습시 그 에러를 표시한 모습